In [47]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data paths
train_path = "CropDisease/train/"
valid_path = "CropDisease/validation/"

# Define data augmentation parameters for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

# Define data augmentation parameters for the validation dataset
valid_datagen = ImageDataGenerator(rescale=1./255)

# Create training and validation data generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Define the CNN  model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(15, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=valid_generator)

# Evaluate the model on the validation dataset
model.evaluate(valid_generator)



Found 1500 images belonging to 15 classes.
Found 1484 images belonging to 15 classes.
Epoch 1/10
47/47 [==============================] - 503s 11s/step - loss: 2.3819 - accuracy: 0.2040 - val_loss: 1.6264 - val_accuracy: 0.4259
Epoch 2/10
47/47 [==============================] - 437s 9s/step - loss: 1.5230 - accuracy: 0.4620 - val_loss: 1.1553 - val_accuracy: 0.5748
Epoch 3/10
47/47 [==============================] - 295s 6s/step - loss: 1.1858 - accuracy: 0.5653 - val_loss: 0.7701 - val_accuracy: 0.6988
Epoch 4/10
47/47 [==============================] - 1344s 29s/step - loss: 0.9947 - accuracy: 0.6220 - val_loss: 0.7522 - val_accuracy: 0.7399
Epoch 5/10
47/47 [==============================] - 215s 5s/step - loss: 0.8887 - accuracy: 0.6627 - val_loss: 0.7654 - val_accuracy: 0.7049
Epoch 6/10
47/47 [==============================] - 476s 10s/step - loss: 0.7919 - accuracy: 0.7027 - val_loss: 0.9925 - val_accuracy: 0.6112
Epoch 7/10
47/47 [==============================] - 408s 9s/step

[0.7243689298629761, 0.6893531084060669]

In [50]:
model.save("crop_disease_model.h5")


Accuracy

In [52]:
# Evaluate model on validation data
result = model.evaluate(valid_batches, verbose=1)
print("Accuracy:", result[1])

149/149 [==============================] - 136s 909ms/step - loss: 2.6226 - accuracy: 0.1765
Accuracy: 0.17654986679553986


In [ ]:
Adding VGG16

In [53]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model

# Load the VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the weights of the pre-trained layers
for layer in vgg.layers:
    layer.trainable = False

# Add new trainable layers
x = Flatten()(vgg.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(15, activation='softmax')(x)

# Define the new model
model = Model(inputs=vgg.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_batches, validation_data=valid_batches, epochs=10, verbose=1)

# Save the model
model.save('crop_disease_vgg.h5')

# Evaluate the model
test_loss, test_acc = model.evaluate(valid_batches, verbose=1)
print('Test accuracy:', test_acc)


Epoch 1/10
150/150 [==============================] - 2287s 15s/step - loss: 11.7400 - accuracy: 0.7413 - val_loss: 1.7650 - val_accuracy: 0.9292
Epoch 2/10
150/150 [==============================] - ETA: 0s - loss: 3.4958 - accuracy: 0.8667

ENJOY

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16

# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Conv2D(256, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(15, activation='softmax')
])

# Load the VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the weights of the pre-trained layers
for layer in vgg_model.layers:
    layer.trainable = False

# Add new trainable layers
x = Flatten()(vgg_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(15, activation='softmax')(x)

# Define the combined model
combined_model = Model(inputs=cnn_model.input, outputs=x)

# Compile the model
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = combined_model.fit(train_batches, validation_data=valid_batches, epochs=10, verbose=1)

# Save the model
combined_model.save('crop_disease_combined.h5')

# Evaluate the model
test_loss, test_acc = combined_model.evaluate(valid_batches, verbose=1)
print('Test accuracy:', test_acc)
